# Углубленная работа с классами и встроенные декораторы

### Методы класса


Методы бывают статическими, классовыми и уровня экземпляра класса (будем их называть обычными методами).
* Статический метод создается с декоратором `@staticmethod`,
* классовый – с декоратором `@classmethod`, первым аргументом в него передается `cls` (ссылка на вызываемый класс),
* обычный метод создается без специального декоратора, ему первым аргументом передается `self`.

In [1]:
class Car:
    
    @staticmethod
    def ex_static_method():
        print("static method")
        
    @classmethod
    def ex_class_method(cls):
        print("class method")
        
    def ex_method(self):
        print("method")

Статический и классовый метод можно вызвать, не создавая экземпляр класса, для вызова ex_method() нужен объект:

In [2]:
Car.ex_static_method() # Работает на классе

static method


In [3]:
Car.ex_class_method() # Работает на классе

class method


In [4]:
Car.ex_method() # Работает на экземпляре класса

TypeError: Car.ex_method() missing 1 required positional argument: 'self'

In [5]:
m = Car() # Экземпляр класса
m.ex_method()

method


**Классовые методы** принимают класс в качестве параметра, который принято обозначать как `cls`. В данном случае он указывает на класс `Car`, а не на объект этого класса, `self`.

Методы класса привязаны к самому классу, а не его экземпляру. Они могут менять состояние класса, что отразится на всех объектах этого класса, но не могут менять конкретный объект.

In [6]:
class MyClass:
    class_attr = "Class attribute"

    @classmethod
    def class_method(cls):
        print(cls.class_attr)

MyClass.class_method()  # Class attribute


Class attribute


In [7]:
class Pizza:
    default_topping = "сыр"

    def __init__(self, topping):
        self.topping = topping

    # Классовый метод (создает пиццу с дефолтной начинкой)
    @classmethod
    def classic(cls):
        return cls(cls.default_topping)  # cls() создает новый экземпляр

# Создаем пиццу через классовый метод
pizza = Pizza.classic()
print(pizza.topping)  # Вывод: сыр

сыр


**Статическим методам** не нужен определённый первый аргумент (ни self, ни cls). Их можно воспринимать как методы, которые `не знают, к какому классу относятся`.

Таким образом, статические методы прикреплены к классу лишь для удобства и не могут менять состояние ни класса, ни его экземпляра. То есть статические методы не могут получить доступ к параметрам класса или объекта. Они работают только с теми данными, которые им передаются в качестве аргументов.

In [8]:
class MathHelper:
    # Статический метод (не зависит от класса или экземпляра)
    @staticmethod
    def add(a, b):
        return a + b

# Вызываем без создания объекта
result = MathHelper.add(5, 3)
print(result)  # Вывод: 8

8


**Метод экземпляра класса** это наиболее часто используемый вид методов. Методы экземпляра класса принимают объект класса как первый аргумент, который принято называть `self` и который указывает на сам экземпляр. Количество параметров метода не ограничено.

Используя параметр `self`, мы можем менять состояние объекта и обращаться к другим его методам и параметрам. К тому же, используя атрибут `self.__class__`, мы получаем доступ к атрибутам класса и возможности менять состояние самого класса. То есть методы экземпляров класса позволяют менять как состояние определённого объекта, так и класса.

### Когда что использовать:
* Обычные методы — когда нужен доступ к данным конкретного объекта.

* Классовые методы — когда нужно работать с классом (например, создать экземпляр с особыми параметрами).

* Статические методы — когда функция логически относится к классу, но не требует доступа к его данным.


Давайте рассмотрим более естественный пример и выясним в чем разница между методами.

In [9]:
from datetime import date

class Car:
    def __init__(self, brand, age):
        self.brand = brand
        self.age = age
    
    @staticmethod
    def is_warranty_active(age):
        return age < 3
    
    @classmethod
    def from_production_year(cls, brand, prod_year):
        return cls(brand, date.today().year - prod_year)
      
    def info(self):
        print("Car: " + self.brand)
        print("Age: " + str(self.age))
        if self.is_warranty_active(self.age):
            print("Warranty is ACTIVE")
        else:
            print("Warranty is NOT active")
    
car1 = Car('Subaru', 5)


In [10]:
car1.brand, car1.age

('Subaru', 5)

In [11]:
car2 = Car.from_production_year('Skoda', 2020) # Создаем экземпляр класса Car

In [12]:
car2.brand, car2.age

('Skoda', 5)

In [13]:
Car.is_warranty_active(25)

False

In [14]:
car1.info()

Car: Subaru
Age: 5
Warranty is NOT active


In [15]:
car2.info()

Car: Skoda
Age: 5
Warranty is NOT active


Метод класса - `from_production_year` возвращает нам СОЗДАННЫЙ внутри функции экземпляр класса `Car` с вычисленным возрастом. Т.к. мы не можем внутри класса `Car` вызвать класс `Car`, мы и используем `cls`.

Статический метод - `is_warranty_active` выясняет действительна ли еще гарантия. Как вы видете, он не обращается к возрасту машины в классе, а принимает ее в качестве аргумента - `age`.

Метод экземпляра класса - `info`, через `self` обращается к своим атрибутам, вызывает статическую функцию, передавая туда возраст машины.

Выбор того, какой из методов использовать, может показаться достаточно сложным. Тем не менее с опытом этот выбор делать гораздо проще. Чаще всего **метод класса** используется тогда, когда нужен генерирующий метод, возвращающий объект класса. Как видим, метод класса `from_production_year` используется для создания объекта класса `Car` по году производства машины, а не по указанному возрасту. 

Статические методы в основном используются как вспомогательные функции и работают с данными, которые им передаются.

Итак:
- Методы экземпляра класса получают доступ к объекту класса через параметр `self` и к классу через `self.__class__`.
- Методы класса не могут получить доступ к определённому объекту класса, но имеют доступ к самому классу через `cls`.
- Статические методы работают как обычные функции, но принадлежат области имён класса. Они не имеют доступа ни к самому классу, ни к его экземплярам.

### Уровни доступа атрибута и метода (Инкапсуляция)

В языках программирования Java, C#, C++ можно явно указать для переменной, что доступ к ней снаружи класса запрещен, это делается с помощью ключевых слов (private, protected и т.д.). В Python таких возможностей нет, и любой может обратиться к атрибутам и методам вашего класса, если возникнет такая необходимость. Это существенный недостаток этого языка, т.к. нарушается один из ключевых принципов ООП – инкапсуляция. Хорошим тоном считается, что для чтения/изменения какого-то атрибута должны использоваться специальные методы, которые называются getter/setter, их можно реализовать, но ничего не помешает изменить атрибут напрямую. При этом есть соглашение, что метод или атрибут, который начинается с нижнего подчеркивания, является скрытым, и снаружи класса трогать его не нужно (хотя сделать это можно).

Внесем соответствующие изменения в класс Car:

In [16]:
class Car:
    def __init__(self, brand, doors_num):
        self._brand = brand
        self._doors_num = doors_num
        
    def get_brand(self):
        return self._brand
    
    def set_brand(self, b):
        self._brand = b
        
    def get_doors(self):
        return self._doors_num
    
    def set_doors(self, d):
        self._doors = d
        
    def info(self):
        return "Nice car with " + str(self._doors) + " doors"

In [20]:
bmw = Car('bmw',2)

In [21]:
bmw._doors_num = 5 # дурной тон


In [24]:
bmw.set_brand(3) #правильная инициализация

In [22]:
print(bmw._doors_num) # дурной тон

5


In [23]:
print(bmw.get_doors()) #правильное обращение

5


В приведенном примере для доступа к `_brand` и` _doors_num` используются специальные методы, но ничего не мешает вам обратиться к ним (атрибутам) напрямую.

In [ ]:
mersedes = Car("Mersedes", 6)
mersedes.get_brand()

In [ ]:
mersedes._brand

Если же атрибут или метод начинается с двух подчеркиваний, то тут напрямую вы к нему уже не обратитесь (простым образом). Модифицируем наш класс Car:

In [25]:
class Car:
    def __init__(self, brand, doors_num):
        self.__brand = brand
        self.__doors_num = doors_num
        
    def get_brand(self):
        return self.__brand
    
    def set_brand(self, b):
        self.__brand = b
    
    def get_doors(self):
        return self.__doors_num
    
    def set_doors(self, d):
        self.__doors = d
        
    def info(self):
        return "Nice car with " + str(self.__doors_num) + " doors"

Попытка обратиться к `__brand` напрямую вызовет ошибку, нужно работать только через get_brand():

In [26]:
mersedes = Car("Mersedes", 6)
mersedes.get_brand()

'Mersedes'

In [27]:
mersedes.__brand

AttributeError: 'Car' object has no attribute '__brand'

Но на самом деле это сделать можно, просто этот атрибут теперь для внешнего использования носит название: `_Car__brand`:

In [28]:
mersedes._Car__brand

'Mersedes'

In [29]:
dir(mersedes)

['_Car__brand',
 '_Car__doors_num',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'get_brand',
 'get_doors',
 'info',
 'set_brand',
 'set_doors']

## Полиморфизм

Как уже было сказано во введении в рамках ООП полиморфизм, как правило, используется с позиции переопределения методов базового класса в классе наследнике. Проще всего это рассмотреть на примере. В нашем базовом класс `Car` есть метод `info()`, который печатает сводную информацию по объекту класса `Car` и переопределим этот метод в классе `Truck`, добавим  в него дополнительные данные:

In [31]:
class Car:
    def __init__(self, brand, doors_num):
        self.__brand = brand
        self.__doors_num = doors_num
        
    def get_brand(self):
        return self.__brand
    
    def set_brand(self, b):
        self.__brand = b
    
    def get_doors(self):
        return self.__doors_num
    
    def set_doors(self, d):
        self.__doors = d
        
    def info(self):
        return "Nice car with " + str(self.__doors_num) + " doors"

In [32]:
class Truck(Car):
    
    def __init__(self, brand, doors_num, load_weight, axes):
        super().__init__(brand, doors_num)
        self.__load_weight = load_weight
        self.__axes = axes
        
    def get_load(self):
        return self.__load_weight
    
    def set_load(self, l):
        self.__load_weight = l
        
    def get_axes(self):
        return self.__axes
    
    def set_axes(self, a):
        self.__axes = a
        
    def info(self):
        return "Nice car with " + str(self.get_doors()) + " doors and can carry " + str(self.__load_weight) + " kg of cargo"

Посмотрим, как это работает

In [33]:
audi = Car("Audi", 4)
audi.info()

'Nice car with 4 doors'

In [34]:
scania = Truck("Scania",2,6500,4)
scania.info()

'Nice car with 2 doors and can carry 6500 kg of cargo'

Таким образом, класс наследник может расширять функционал класса родителя.

## Наследование

Возможность одному классу выступать в качестве наследника для другого, перенимая тем самым его свойства и методы, является важной особенностью ООП. Благодаря этой важной особенности пропадает необходимость переписывания кода для подобных или родственных по назначению классов.

При наследовании классов в Python обязательно следует соблюдать одно условие: `класс-наследник` должен представлять собой более частный случай `класса-родителя`. В следующем примере показывается как класс `Car` наследуется классом `Truck`. При описании подкласса в Python, имя родительского класса записывается в круглых скобках.

In [35]:
class Car:
    def __init__(self, brand, doors_num):
        self.__brand = brand
        self.__doors_num = doors_num
        
    def get_brand(self):
        return self.__brand
    
    def set_brand(self, b):
        self.__brand = b
    
    def get_doors(self):
        return self.__doors_num
    
    def set_doors(self, d):
        self.__doors = d
        
    def info(self):
        return "Nice car with " + str(self.__doors_num) + " doors"

In [36]:
class Truck(Car):
    
    def __init__(self, brand, doors_num, load_weight, axes):
        super().__init__(brand, doors_num)
        self.__load_weight = load_weight
        self.__axes = axes
    
    def get_load(self):
        return self.__load_weight
    
    def set_load(self, l):
        self.__load_weight = l
        
    def get_axes(self):
        return self.__axes
    
    def set_axes(self, a):
        self.__axes = a

Родительским классом является `Car`, который при инициализации принимает бренд машины и количество дверей и предоставляет его через свойства. `Truck` – класс наследник от `Car`. Обратите внимание на его метод `__init__`: в нем первым делом вызывается конструктор его родительского класса: `super().__init__(brand, doors_num)`

`super` – это ключевое слово, которое используется для обращения к родительскому классу. Теперь у объекта класса `Truck` помимо уже знакомых свойств `brand` и `doors_num` появились свойства `load_weight` и `axes`:

In [37]:
truck = Truck("Kamaz",2,13000,6)

truck.get_brand()

'Kamaz'

И смотрите, методы из родительского класса работают!

In [38]:
truck.get_load()

13000

In [39]:
truck.set_axes(8)
truck.get_axes()

8

In [42]:
truck.get_brand()

'Kamaz'

In [43]:
truck.info()

'Nice car with 2 doors'

In [40]:
dir(truck)

['_Car__brand',
 '_Car__doors_num',
 '_Truck__axes',
 '_Truck__load_weight',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'get_axes',
 'get_brand',
 'get_doors',
 'get_load',
 'info',
 'set_axes',
 'set_brand',
 'set_doors',
 'set_load']

Проблема ромбовидного наследования возникает, когда производный класс наследует от нескольких классов, которые имеют общего предка. Для разрешения этой проблемы используется MRO (Method Resolution Order) - порядок разрешения методов.

In [44]:

class A:
    def method(self):
        print("A method")

class B(A):
    def method(self):
        print("B method")

class C(A):
    def method(self):
        print("C method")

class D(B, C):
    pass

obj = D()
obj.method()  # B method


B method


### Объяснение:

#### Ромбовидное наследование:

* Классы B и C наследуют от A

* Класс D наследует от B и C

* Иерархия образует ромб: A → B, A → C → D

#### Проблема:

* Если вызвать d.show(), от какого класса (B, C или A) будет взят метод?

* Без четкого порядка разрешения методов возможна неоднозначность.

#### Решение через MRO (Method Resolution Order):

* Python использует алгоритм C3 для определения порядка поиска методов.

* MRO можно посмотреть через D.__mro__ или D.mro()

In [45]:
print(D.mro()) 
# Вывод: [D, B, C, A, object]

[<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>]


### Идиома для вызова конструктора текущего класса:


Иногда требуется вызвать конструктор текущего класса вместо конструктора родительского класса. Это может быть полезно, например, при создании подкласса с расширенным функционалом, но с сохранением базового состояния.




In [ ]:
class MyBaseClass:
    def __init__(self, x):
        self.x = x

class MySubClass(MyBaseClass):
    def __init__(self, x, y):
        super().__init__(x)
        self.y = y

obj = MySubClass(1, 2)
print(obj.x)  # 1
print(obj.y)  # 2

### Класс object:

object является базовым классом для всех классов в Python. Все классы неявно наследуют от класса object и получают его функциональность. Класс object определяет некоторые магические методы, такие как __str__, __repr__, __eq__, которые можно переопределить в производных классах для изменения их поведения.




In [ ]:
class MyClass:
    pass

obj = MyClass()
print(obj.__str__())  # <__main__.MyClass object at 0x000001>
print(obj.__repr__())  # <__main__.MyClass object at 0x000001>

In [ ]:
dir(obj)

### Множественное наследование

Наследовать можно не только один класс, но и несколько одновременно, обретая тем самым их свойства и методы. В данном примере класс `Dog` выступает в роли подкласса для `Animal` и `Pet` , поскольку может являться и тем, и другим. От `Animal Dog` получает способность спать (метод `sleep`), в то время как `Pet` дает возможность играть с хозяином (метод `play`). В свою очередь, оба родительских класса унаследовали поле `name` от `Creature`. Класс `Dog` также получил это свойство и может его использовать. Так как мы не используем конструкторы в наследованных классах, то и вызывать через `super()` ничего не надо. Конструктор родительского класса, вызовется автоматически.

In [46]:
class Creature:
    def __init__(self, name):
        self.name = name
        
class Animal(Creature):
    def sleep(self):
        print(self.name + " is sleeping")
        
class Pet(Creature):
    def play(self):
        print(self.name + " is playing")
        
class Dog(Animal, Pet):
    def bark(self):
        print(self.name + " is barking")
        
beast = Dog("Buddy")
beast.sleep()
beast.play()
beast.bark()

Buddy is sleeping
Buddy is playing
Buddy is barking


In [47]:
dir(beast)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'bark',
 'name',
 'play',
 'sleep']

В вышеописанном примере создается объект класса `Dog`, получающий имя в конструкторе. Затем по очереди выполняются методы `sleep`, `play` и `bark`, двое из которых были унаследованы. Способность лаять является уникальной особенностью собаки, поскольку не каждое животное или домашний питомец умеет это делать.

## Абстрактные методы

Поскольку в ООП присутствует возможность наследовать поведение родительского класса, иногда возникает необходимость в специфической реализации соответствующих методов. В качестве примера можно привести следующий код, где классы `Truck` и `Bus` являются потомками класса `Car`. Как и положено, они оба наследуют метод `horn` (гудеть), однако в родительском классе для него не существует реализации.

Все потому, что машина представляет собой абстрактное понятие, а значит она не способна издавать какой-то конкретный гудок. Однако для грузовика и автобуса данная команда зачастую имеет общепринятое значение. В таком случае можно утверждать, что метод `honk` из `Car` является абстрактным, поскольку не имеет собственного тела реализации.

In [48]:
class Car:
    def __init__(self, brand):
        self.__brand = brand
        
    def honk(self):
        pass
    
class Truck(Car):
    def honk(self):
        print("RRRRrrrr")
        
class Bus(Car):
    def honk(self):
        print("UUUUUU")
        
        
Vanhool = Bus("Vanhool")
Iveco = Truck("Iveco")

Vanhool.honk()
Iveco.honk()

UUUUUU
RRRRrrrr


Как видно из примера, потомки `Truck` и `Bus` получают `horn`, после чего переопределяют его каждый по-своему. В этом заключается суть полиморфизма, позволяющего изменять ход работы определенного метода исходя из нужд конкретного класса. При этом название у него остается общим для всех наследников, что помогает избежать путаницы с именами.

Начиная с версии языка 2.6 в стандартную библиотеку включается модуль abc, добавляющий в язык абстрактные базовые классы (далее АБК).

АБК позволяют определить класс, указав при этом, какие методы или свойства обязательно переопределить в классах-наследниках:

In [49]:
from abc import ABC, abstractmethod

class Book(ABC):
    def __init__(self, title, author):
        self.title = title
        self.author = author

    @abstractmethod
    def get_summary(self):
        pass

class Fiction(Book):
    def get_summary(self):
        print(f'"{self.title}" - роман в стиле исторический фикшн, автор - {self.author}')

class NonFiction(Book):
    def get_summary(self):
        print(f'"{self.title}" - книга в стиле нон фикшн, автор - {self.author}')

class Poetry(Book):
    pass

Класс Book имеет абстрактный метод get_summary(). Два подкласса Book (Fiction и NonFiction) реализуют метод get_summary(), а третий подкласс Poetry – нет. Когда мы создаем экземпляры Fiction и NonFiction и вызываем их методы get_summary(), получаем ожидаемый результат:

In [50]:
fiction_book = Fiction("Террор", "Дэн Симмонс")
nonfiction_book = NonFiction("Как писать книги", "Стивен Кинг")
fiction_book.get_summary()
nonfiction_book.get_summary()

"Террор" - роман в стиле исторический фикшн, автор - Дэн Симмонс
"Как писать книги" - книга в стиле нон фикшн, автор - Стивен Кинг


А вот вызов Poetry приведет к ошибке, поскольку в этом подклассе метод get_summary() не реализован:

In [51]:
poetry_book = Poetry("Стихотворения", "Борис Пастернак")

TypeError: Can't instantiate abstract class Poetry without an implementation for abstract method 'get_summary'

In [ ]:
len(dir(nonfiction_book))

In [ ]:
len(dir(fiction_book))

### Создание пользовательских исключений:

Python позволяет создавать собственные исключения для обработки специфических ситуаций в коде. Для создания пользовательского исключения необходимо определить новый класс, наследующийся от класса Exception или его подклассов.


🤓Можем написать свое исключение для класса комплексных чисел или для векторов, например. Пускай мы будем бросать исключение VectorError, если хотим сложить вектора разной длины.


In [52]:
class CustomException(Exception):
    pass

try:
    raise CustomException("This is a custom exception")
except CustomException as e:
    print(e)  # This is a custom exception

This is a custom exception


## Практика

Создать систему наследования для геометрических фигур: фигура, прямоугольник, квадрат, круг (можно добавить и другие плоские фигуры - например, параллелограмм, треугольник, прямоугольный треугольник). 
У всего определить площадь и периметр, а также `__str__` (дополнительно можно добавить `__repr__`). 

**Важно**: где-то нужно переопределять методы, а площадь абстрактной фигуры не определена.

In [ ]:
from math import pi

class Figure:
    def area(self):
        raise NotImplementedError("Площадь не определена для абстрактной фигуры")

    def perimeter(self):
        raise NotImplementedError("Периметр не определен для абстрактной фигуры")

    def __str__(self):
        return "Это геометрическая фигура"

class Rectangle(Figure):
    def __init__(self, width, height):
        self.width = width
        self.высота = height

    def area(self):
        return self.width * self.height

    def perimeter(self):
        return 2 * (self.width + self.width)

    def __str__(self):
        return f"Прямоугольник шириной {self.width} и высотой {self.height}"

class Square(Rectangle):
    def __init__(self, side):
        super().__init__(side, side)

    def __str__(self):
        return f"Квадрат со стороной {self.width}"

class Cycle(Figure):
    def __init__(self, radius):
        self.radius = radius

    def area(self):
        return pi * self.radiusс ** 2

    def perimeter(self):
        return 2 * pi * self.radius

    def __str__(self):
        return f"Круг с радиусом {self.radius}"


### Полезные материалы
1. Абстрактные классы в Python http://pythonicway.com/education/python-oop-themes/33-python-abstract-class
2. Наследование  https://metanit.com/python/tutorial/7.3.php 

### Вопросы для закрепления
1. Зачем нужны приватные поля? Как эта идея реализована в Python?
2. Что такое полиморфизм? Зачем он нужен? Приведите пример
3. Зачем могут понадобиться абстрактные классы? Приведите пример задачи, в которой нужен класс, экземляр которого не нужно создавать ибо он не имеет смысла
